# Document Processing and Embedding with ChromaDB and Sentence Transformers

In [5]:
import chromadb
from sentence_transformers import SentenceTransformer

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import os

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

client = chromadb.PersistentClient(path="../db/pdfs")
md_collection = client.get_or_create_collection("markdown_chunks_collection")

def retrieve_relevant_chunks(query, collection, model, top_k=5):
    # Encode the query using the same SentenceTransformer model
    query_embedding = model.encode(query)

    # Query the ChromaDB collection for the top_k similar chunks
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=top_k,
    )
    
    # Extract the documents from the query results
    # relevant_chunks = results["documents"][0]  # Since there's only one query
    # return relevant_chunks
    return results



# Create system-level prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create a question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Function to generate a response using retrieval
def get_response(query, collection, model, question_answer_chain):
    # Retrieve relevant chunks from ChromaDB
    relevant_chunks = retrieve_relevant_chunks(query, collection, model)
    
    # Combine the retrieved chunks into a single context string
    context = "\n".join(relevant_chunks)
    
    # Invoke the QA chain with the provided context
    response = question_answer_chain.invoke({
        "context": context,
        "input": query
    })
    
    return response

# Run a query
query = "What is Insertion Sort?"

#Load the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

results = retrieve_relevant_chunks(query, md_collection, model)
print(results)
# for i, document in enumerate(results["documents"][0]):
#     print(f"Result {i+1}: {document}")
#     print(f"ID: {results['ids'][0][i]}")
    # print(f"Metadata: {results['metadatas'][0][i]}")
    # print(f"Similarity score: {results['distances'][0][i]}\n")

{'ids': [['815', '831', '818', '525', '526']], 'embeddings': None, 'documents': [['##### 9.5 Insertion Sort\n\n_Insertion Sort is (not surprisingly) a form of insertion sorting. It starts by treating the first_\nentry a[0] as an already sorted array, then checks the second entry a[1] and compares it with\nthe first. If they are in the wrong order, it swaps the two. That leaves a[0],a[1] sorted.\nThen it takes the third entry and positions it in the right place, leaving a[0],a[1],a[2]\nsorted, and so on. More generally, at the beginning of the ith stage, Insertion Sort has the\nentries a[0],..., a[i-1] sorted and inserts a[i], giving sorted entries a[0],...,a[i].\nFor the example starting array 4 1 3 2, Insertion Sort starts by considering a[0]=4\nas sorted, then picks up a[1] and ‘inserts it’ into the already sorted array, increasing the size\nof it by 1. Since a[1]=1 is smaller than a[0]=4, it has to be inserted in the zeroth slot,\n\n67\n\n|4|1|3|2|\n|---|---|---|---|\n\n\n-----', '*

# 

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_chroma import Chroma
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List
import os
from langchain_openai import ChatOpenAI

class MyEmbeddings(Embeddings):
        def __init__(self):
            self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode(query).tolist()

embedding_func = MyEmbeddings()

# Initialize Ollama LLM
llm = OllamaLLM(
    # model="gemma2:2b",
    model = "llama3.2:latest",
    base_url="http://localhost:11434"  # Adjust this URL if needed
)

# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     api_key=os.environ.get("OPENAI_API_KEY"),
#     # api_key=st.secrets["OpenAI_key"]
# )

from langchain_core.prompts import ChatPromptTemplate

# Define a prompt template for Ollama to generate keywords
# Gemma2:2b Template
# keyword_prompt_template = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are an assistant that generates keywords for a chunk of text. The keywords must be single words or two-word phrases. Format the output as: ['keyword1', 'keyword2']"),
#         ("human", "Extract relevant keywords for the following chunk:\n\n{chunk_text}")
#     ]
# )

# Llama3.2:1b Template
keyword_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an assistant that generates keywords for a chunk of text. "
                   "Your response should only be a Python list of keywords with no introductory text, in the format: ['keyword1', 'keyword2']"),
        ("human", "Extract relevant keywords for the following chunk:\n\n{chunk_text}")
    ]
)

chain = keyword_prompt_template | llm

# Initialize ChromaDB client
vector_store = Chroma(
    # client = client,
    collection_name="markdown_chunks_collection",
    embedding_function=embedding_func,
    persist_directory = "../db/test_pdfs",
    # other params...
)

def split_chunks():
    try:
        from pathlib import Path
        from langchain_core.documents import Document
        from langchain_text_splitters import RecursiveCharacterTextSplitter as Rec
        
        # Path to markdown directory
        md_dir = Path("../data/md/")
        chunk_id_counter = 1  # Initialize a counter for unique chunk IDs
        ids = []
        documents = []

        # Loop through all markdown files in the md directory
        for md_file in md_dir.glob("*.md"):
            with open(md_file, "r") as f:
                md_content = f.read()

            # Chunk the markdown content
            text_splitter = Rec(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len,
                add_start_index=True
            )
            chunks = text_splitter.split_text(md_content)
            
            for chunk in chunks:
                # Generate keywords for the chunk using Ollama
                response = chain.invoke({"chunk_text": chunk})
                # keywords = response["choices"][0]["message"]["content"].strip()
                keywords = response.strip("[]").replace("'", "").split(", ")
                print(keywords)
                
                # Create a Document object with metadata for the chunk, including keywords
                document_to_add = Document(
                    page_content=chunk,
                    metadata={"source": str(md_file), "keywords": str(keywords)}
                )
                
                documents.append(document_to_add)
                ids.append(str(chunk_id_counter))  # Add document ID to the list
                chunk_id_counter += 1  # Increment the ID counter
        
        # Assuming vector_store is defined and initialized elsewhere
        vector_store.add_documents(documents=documents, ids=ids)
    except Exception as e:
        print(f"Error: {e}")
        
if __name__ == "__main__":
    split_chunks()


# Multi Query

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain.embeddings.base import Embeddings
from typing import List
import os
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from sentence_transformers import SentenceTransformer

class MyEmbeddings(Embeddings):
        def __init__(self):
            self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode(query).tolist()

embedding_func = MyEmbeddings()


# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

gpt4 = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    # api_key=st.secrets["OpenAI_key"],
    temperature=0
)

# llm = OllamaLLM(model="gemma2:2b", base_url="http://localhost:11434")
llm = OllamaLLM(model="gemma2:2b", base_url="http://localhost:11434")

generate_queries = (
    prompt_perspectives 
    | gpt4
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Initialize ChromaDB client
vector_store = Chroma(
    # client = client,
    collection_name="markdown_chunks_collection",
    embedding_function=embedding_func,
    persist_directory = "../db/pdfs",
    # other params...
)

retriever = vector_store.as_retriever()

# Retrieve
question = "What is Insertion Sort?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
# print(docs)
len(docs)

from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | gpt4
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})


c:\Users\elroy\anaconda3\envs\chatbot_20241028\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\elroy\AppData\Local\Temp\ipykernel_15096\1701805498.py:58: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


'Insertion Sort is a sorting algorithm that builds a sorted array (or list) one element at a time. It is similar to the way one might sort a hand of playing cards. The algorithm works by taking one element from the unsorted portion of the array and inserting it into the correct position in the sorted portion, ensuring that the sorted portion remains in order at each step.\n\nThe process can be summarized as follows:\n1. Start with an initially empty sorted list.\n2. Take each item from the unsorted list and find the appropriate position in the sorted list.\n3. Shift elements in the sorted list as necessary to make room for the new item.\n4. Repeat this process until all items have been sorted.\n\nInsertion Sort has a time complexity of O(n²) in the average and worst cases, making it less efficient for large datasets compared to more advanced algorithms. However, it is stable (it preserves the relative order of items with equal keys) and can be efficient for small datasets or partially 

# RAG-Fusion

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain.embeddings.base import Embeddings
from typing import List
import os
from langchain_openai import ChatOpenAI
from sentence_transformers import SentenceTransformer

class MyEmbeddings(Embeddings):
        def __init__(self):
            self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode(query).tolist()

embedding_func = MyEmbeddings()


# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    # api_key=st.secrets["OpenAI_key"],
    temperature=0
)

generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

# Initialize ChromaDB client
vector_store = Chroma(
    # client = client,
    collection_name="markdown_chunks_collection",
    embedding_function=embedding_func,
    persist_directory = "../db/pdfs",
    # other params...
)

retriever = vector_store.as_retriever()

# Retrieve
question = "What is Insertion Sort?"
retrieval_chain_rag_fusion  = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain.invoke({"question":question})
# print(docs)
len(docs)

from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})


'Insertion Sort is a sorting algorithm that builds a sorted array (or list) one element at a time. It starts by treating the first entry as an already sorted array and then compares the next entry with the sorted portion. If the next entry is smaller than the sorted entries, it is inserted into the correct position by shifting the larger entries up to make space. This process continues for each subsequent entry until the entire array is sorted.\n\nThe general algorithm for Insertion Sort can be described as follows:\n\n1. Start with the first element as sorted.\n2. For each subsequent element, compare it with the elements in the sorted portion.\n3. Shift the larger elements up to make space for the new element.\n4. Insert the new element into its correct position.\n5. Repeat until all elements are sorted.\n\nInsertion Sort is considered stable, meaning that it preserves the relative order of items with equal keys. Its average and worst-case time complexity is O(n²), making it less effi

In [3]:
from utils.bm25_ranking import find_closest_chunks_bm25, new_bm25, re_rank_chunks_with_embeddings
print("OLD BM25")
results_top_n = find_closest_chunks_bm25(query, results2, top_n=10)
for res in results_top_n:
    print(res['score']," ",res['id'],"\n")
    # print(res['document'],"\n")


print("NEW BM25")
bm25_results = new_bm25(query, results2, top_n=10)
sorted_results = re_rank_chunks_with_embeddings(query, bm25_results)

# print(results_top_n)
# for res in results_top_n:
#     print(res['score']," ",res['id'],"\n")
#     print(res['document'],"\n")

# organised_list = {}
# for item in results2:
#     print(item)
#     # organised_list[results2[item]] = ## id and document
# print(organised_list)
# print(results_top_n)
# print(md_collection.get(ids="297"))

OLD BM25
14.584834019480919   377 

12.523017371922746   378 

12.050891594269249   375 

11.880403985989389   866 

11.260478820127473   535 

11.123203021744485   815 

11.035082004738484   814 

11.002614755875888   406 

10.965935890038589   260 

10.352130470797842   805 

NEW BM25
Chunk ID: 815, BM25 Score: 11.123203021744485, Embedding Score: 0.4674035310745239, Source: data\md\dsa.md
Most libraries provide implementations of unordered sets and so DSA does
not; we simply mention it here to disambiguate between an unordered set and
ordered set.

We will only look at insertion for an unordered set and cover briefly why a
hash table is an efficient data structure to use for its implementation.

###### 5.1.1 Insertion

An unordered set can be efficiently implemented using a hash table as its backing
data structure. As mentioned previously we only add an item to a set if that
item is not already in the set, so the backing data structure we use must have
a quick look up and insertion 

In [22]:
print(md_collection.get(ids=['220']))

{'ids': ['220'], 'embeddings': None, 'metadatas': [{'source': 'data\\md\\cp1.md'}], 'documents': ['best possible acorns collected when Jayjay is at this height. The bottom-up DP code that requires\n\nonly 2000 = 2K states and time complexity of 2000 2000 = 4M is as follow:\n_×_\n\nfor (int tree = 0; tree < t; tree++) // initialization\ndp[h] = max(dp[h], acorn[tree][h]);\nfor (int height = h - 1; height >= 0; height--)\nfor (int tree = 0; tree < t; tree++) {\nacorn[tree][height] +=\nmax(acorn[tree][height + 1], // from this tree, +1 above\n((height + f <= h) ? dp[height + f] : 0)); // best from tree at height + f\ndp[height] = max(dp[height], acorn[tree][height]); // update this too\n}\nprintf("%d\\n", dp[0]); // solution will be here\n\nLesson: When na¨ıve DP states are too large causing the overall DP time complexity not-doable,\n\nthink of different ways other than the obvious to represent the possible states. Remember that no\n\nprogramming contest problem is unsolvable, the proble